# Torus one-point functions of the $O(n)$ model

In [1]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


Number of threads: 4


In [2]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=sqrt(big"2")/(big"0.8"+big"0.2"*im))
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2*precision), base=10)
    Δmax = floor(Int, 0.75*precision)
    co = Correlation(Field(c, r=indices[1], s=indices[2]), Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
        specs, signature;
        rmax=3, show=true, 
        Ps=nothing, diagchan=nothing,
        fix=nothing
    )
    specs = @channels filter(
        V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan],
        specs[chan]
    )
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix)
        if show
            Base.show(stdout, res.str_cst, rmax=rmax)
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end

solve (generic function with 1 method)

## r = 0

In [ ]:
ind = (0, sqrt(big"2")*(big"0.4"+big"0.2"*im)) # P generic
blocks = precompute_blocks(ind, fields, parity=1, precision=20)
sol = solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬──────────────────────┬───────────┐
│ Field           │ Structure constant   │ Rel. err. │
├─────────────────┼──────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im            │ 0         │
│ (1//2, 0)       │ 0.407941+0.116402im  │ 0.014     │
│ (1, 0)          │ -0.983026+0.176574im │ 0.4       │
│ (1, 1)          │ 0.378244+0.35468im   │ 0.2       │
│ (3//2, 0)       │ -2678.48-412.949im   │ 1         │
│ (3//2, 2//3)    │ -8083.39-8332.89im   │ 0.17      │
└─────────────────┴──────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────────┬───────────┐
│ Field           │ Structure constant     │ Rel. err. │
├─────────────────┼────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 0.999804-0.000999659im │ 0.00079   │
│ (1//2, 0)       │ 0.41814+0.117177im     │ 0.0033    │
│ (1, 0)          │ 0.303868-0.335174im    │ 0.15      │
│ (1, 1)          │ -0.0255678-0.480167im  │ 0.2       │
│ (3//2, 0)       │ -79.9418-

In [5]:
β = c.β
ind = (0, β * (β - 2/β)) # P = P_(1, 2)
blocks = precompute_blocks(ind, fields, parity=1, precision=30)
solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼─────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │               1.0+0.0im │            0.0 │
│       (1//2, 0) │      0.355233-0.12359im │    9.07256e-31 │
│          (1, 0) │     0.202314-0.121132im │    9.18332e-28 │
│          (1, 1) │  -0.0591978-0.0718335im │    5.64661e-28 │
│       (3//2, 0) │   0.0700839-0.0659127im │    9.43259e-23 │
│    (3//2, 2//3) │   -0.144613-0.0452004im │    3.39973e-23 │
│          (2, 0) │ 0.0413411+0.000518487im │    5.01249e-16 │
│       (2, 1//2) │   -0.110954+0.0600867im │    1.41334e-16 │
│          (2, 1) │  -0.143968+0.00561456im │    1.17419e-16 │
└─────────────────┴─────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼──────────────

## $r = 1, s = 0$

In [32]:
blocks = precompute_blocks((1, 0), fields, parity=1, precision=80);

### Signature $(0, 0, 0)$

In [53]:
using BarnesDoubleGamma
setprecision(BigFloat, 20, base=10)
β = c.β
DG = DoubleGamma(β)
V = Field(c, r=3//2, s=2//3)
V1 = Field(c, r=1, s=0)
V2 = Field(c, r=0, s=β^2)
println(V1[:left].Δ ≈ V2[:left].Δ)
println(LoopModels.Cref(V2, V, V) / LoopModels.Cref(V1, V, V))
println(gamma(1))

true
2.000000000630581425861 + 6.185447576316674431699e-10im
1.0


In [72]:
ϵ = 1e-8
V0 = Field(c, r=0, s=1//2)
V = Field(c, r=1, s=0)
V1 = Field(c, r=1, s=0)
V2 = Field(c, r=0, s=β^2)
println(LoopModels.Cref(V2, V0, V) / LoopModels.Cref(V1, V0, V))
println(LoopModels.Bref(V1, DG))
println(2 / DG(2β)^2 / DG(β + 1 / β)^2)

1.000000000000000000027 + 0.0im
101.1813198816819643318 + 28.95437238763036572509im
101.1801640727629699812 + 28.95462018026876427779im


In [23]:
# show(stdout, sol.str_cst, backend=:latex, rmax=3)

## $r=1, s = 2$

In [ ]:
blocks = precompute_blocks((1, 2), fields, parity=0, precision=60);
sol = solve(blocks, Sig(0, 0, 0));

 Channel s
┌─────────────────┬──────────────────────┬────────────────┐
│ Field           │ Structure constant   │ Relative error │
├─────────────────┼──────────────────────┼────────────────┤
│ (P=0.11+0.06im) │            1.0+0.0im │            0.0 │
│       (1//2, 0) │   -4.40242+4.03791im │    1.01343e-29 │
│          (1, 0) │   -10.1201+11.9725im │    3.31304e-27 │
│          (1, 1) │    7.34646+7.71471im │    7.29649e-27 │
│       (3//2, 0) │   -6.96244+12.7403im │    7.16963e-23 │
│    (3//2, 2//3) │    29.3043+3.40064im │    9.87459e-23 │
│   (3//2, -2//3) │    17.0082-3.12202im │    6.41755e-23 │
│          (2, 0) │   -10.6338+3.27378im │    4.17103e-16 │
│      (2, -1//2) │    18.1994-22.7161im │    1.74474e-16 │
│       (2, 1//2) │    29.2332-26.2693im │    4.09395e-16 │
│          (2, 1) │     50.421-10.7106im │     3.4283e-16 │
│       (5//2, 0) │   -64.0131-14.2118im │     9.39071e-8 │
│   (5//2, -2//5) │   0.148746-30.5888im │      1.2882e-7 │
│    (5//2, 2//5) │    4.0134

In [26]:
# compare with V_{P_{(1, 2)}}
β = c.β
blocks = precompute_blocks((0, β * (β - 2/β)), fields, parity=0, precision=30)
solve(blocks, Sig(0, 0, 0))

 Channel s
┌─────────────────┬─────────────────────────┬────────────────┐
│ Field           │ Structure constant      │ Relative error │
├─────────────────┼─────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │               1.0+0.0im │            0.0 │
│       (1//2, 0) │      0.355233-0.12359im │    2.49264e-30 │
│          (1, 0) │     0.202314-0.121132im │    1.32465e-27 │
│          (1, 1) │  -0.0591978-0.0718335im │    2.37294e-27 │
│       (3//2, 0) │   0.0700839-0.0659127im │    1.07924e-22 │
│    (3//2, 2//3) │   -0.144613-0.0452004im │    4.93023e-23 │
│   (3//2, -2//3) │   -0.144613-0.0452004im │    2.87732e-23 │
│          (2, 0) │ 0.0413411+0.000518487im │    7.49363e-16 │
│      (2, -1//2) │   -0.110954+0.0600867im │    1.66018e-16 │
│       (2, 1//2) │   -0.110954+0.0600867im │    1.58187e-16 │
│          (2, 1) │  -0.143968+0.00561456im │    1.22106e-16 │
│       (5//2, 0) │     0.13455+0.0783828im │     1.49143e-7 │
│   (5//2, -2//5) │  -0.0288292+0.0747277im 

BootstrapSystem{Complex{BigFloat}}
< (P=0.27+-0.35im) >
Number of positions: 58
Matrix size: (116, 110)


## r = 2

In [32]:
blocks = precompute_blocks((2, 0), fields, parity=1, precision=25);

### Signature $(\frac12, \frac12, \frac12)$

In [21]:
solve(blocks, Sig(1//2, 1//2, 1//2), rmax=3//2);

 Channel s
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relative error │
├──────────────┼────────────────────┼────────────────┤
│    (1//2, 0) │          1.0+0.0im │            0.0 │
│       (1, 0) │ -34.9208-45.5559im │     3.6251e-13 │
│       (1, 1) │  31.0888+25.0735im │    3.59439e-13 │
│    (3//2, 0) │ -589.454+35.4061im │    6.61775e-10 │
│ (3//2, 2//3) │  277.297-26.5527im │     1.33174e-9 │
└──────────────┴────────────────────┴────────────────┘
 Channel t
┌──────────────┬────────────────────┬────────────────┐
│ Field        │ Structure constant │ Relative error │
├──────────────┼────────────────────┼────────────────┤
│    (1//2, 0) │  1.0+4.58897e-14im │    1.54374e-14 │
│       (1, 0) │ -34.9208-45.5559im │    6.25701e-14 │
│       (1, 1) │  31.0888+25.0735im │    1.25781e-13 │
│    (3//2, 0) │ -589.454+35.4061im │    1.15658e-11 │
│ (3//2, 2//3) │  277.297-26.5527im │    5.00024e-11 │
└──────────────┴────────────────────┴──────

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = 0$

In [36]:
fix = [(:s, diag_field, 1.0), (:s, Field(c, r=1//2, s=0), 0.0)]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3//2);
println("With $(length(fix)-1) constant fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=1//2);

With 2 constants fixed:
 Channel s
┌─────────────────┬────────────────────┬────────────────┐
│ Field           │ Structure constant │ Relative error │
├─────────────────┼────────────────────┼────────────────┤
│ (P=0.11+0.06im) │          1.0+0.0im │            0.0 │
│       (1//2, 0) │          0.0+0.0im │            0.0 │
│          (1, 0) │ -934.438+210.945im │     1.0196e-20 │
│          (1, 1) │ -597.189+296.177im │    4.18608e-21 │
│       (3//2, 0) │  1538.45+4136.88im │    6.37391e-17 │
│    (3//2, 2//3) │  1587.23+3847.92im │    2.70553e-17 │
└─────────────────┴────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────┬────────────────┐
│ Field           │ Structure constant │ Relative error │
├─────────────────┼────────────────────┼────────────────┤
│ (P=0.11+0.06im) │   1.0-1.0416e-20im │    3.28862e-21 │
│       (1//2, 0) │                  0 │       0.426329 │
│          (1, 0) │ -934.438+210.945im │    3.15695e-21 │
│          (1, 1) │ -597.1

We find that the space of solutions has dimension 2.

## r = 3

In [39]:
blocks = precompute_blocks((3, 0), fields, parity=1, precision=20);

### Signature $(1, 1, 1)$

In [24]:
solve(blocks, Sig(1, 1, 1), rmax=2);

 Channel s
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │            1.0+0.0im │            0.0 │
│       (1, 1) │ -1.12641-0.0491036im │    8.92952e-14 │
│    (3//2, 0) │                    0 │       0.601382 │
│ (3//2, 2//3) │                    0 │        1.34188 │
│       (2, 0) │    10728.0+15111.3im │        1.17e-6 │
│    (2, 1//2) │   -10603.0-14053.7im │     1.04044e-6 │
│       (2, 1) │    10112.1+11149.5im │     9.93827e-7 │
└──────────────┴──────────────────────┴────────────────┘
 Channel t
┌──────────────┬──────────────────────┬────────────────┐
│ Field        │ Structure constant   │ Relative error │
├──────────────┼──────────────────────┼────────────────┤
│       (1, 0) │    1.0-5.91106e-14im │    8.71426e-14 │
│       (1, 1) │ -1.12641-0.0491036im │    1.73418e-13 │
│    (3//2, 0) │                    0 │        1.90731 │
│ (3//2, 

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = D^{(s)}_{(1, 0)} = 0$

In [40]:
fix = [
    (:s, diag_field, 1),
    (:s, Field(c, r=1//2, s=0), 0),
    (:s, Field(c, r=1, s=0), 0)
]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3//2);
println("With $(length(fix)-1) constant fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=1);

With 3 constants fixed:
 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │                  1.0+0.0im │            0.0 │
│       (1//2, 0) │                  0.0+0.0im │            0.0 │
│          (1, 0) │                  0.0+0.0im │            0.0 │
│          (1, 1) │          249.309-34.0451im │    5.75536e-12 │
│       (3//2, 0) │ -1.17386e+06-5.49204e+06im │      1.124e-12 │
│    (3//2, 2//3) │ -1.46418e+06-4.26679e+06im │    1.10834e-12 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.11+0.06im) │          1.0-3.12744e-13im │    2.65427e-13 │
│       (1//2, 0) │           

The space of solutions has dimension $3 = |\mathcal{M}^c_{1, 1}(3)|$.

## r = 4

In [46]:
blocks = precompute_blocks((4, 0), fields, parity=1, precision=20);

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = D^{(s)}_{(1, 0)} = D^{(s)}_{(1, 1)} = 0$

In [47]:
fix = [
    (:s, diag_field, 1),
    (:s, Field(c, r=1//2, s=0), 0),
    (:s, Field(c, r=1, s=0), 0),
    (:s, Field(c, r=1, s=1), 0)
]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3//2);
println("With $(length(fix)-1) constant fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=1);

With 4 constants fixed:
 Channel s
┌─────────────────┬────────────────────┬────────────────┐
│ Field           │ Structure constant │ Relative error │
├─────────────────┼────────────────────┼────────────────┤
│ (P=0.11+0.06im) │          1.0+0.0im │            0.0 │
│       (1//2, 0) │          0.0+0.0im │            0.0 │
│          (1, 0) │          0.0+0.0im │            0.0 │
│          (1, 1) │          0.0+0.0im │            0.0 │
│       (3//2, 0) │  32440.3+62680.7im │     1.29247e-7 │
│    (3//2, 2//3) │ -16345.9-30713.7im │     2.72176e-7 │
└─────────────────┴────────────────────┴────────────────┘
 Channel t
┌─────────────────┬────────────────────┬────────────────┐
│ Field           │ Structure constant │ Relative error │
├─────────────────┼────────────────────┼────────────────┤
│ (P=0.11+0.06im) │  1.0+3.10777e-10im │    2.05216e-10 │
│       (1//2, 0) │                  0 │        1.74194 │
│          (1, 0) │                  0 │        1.58961 │
│          (1, 1) │       

The space of solutions has dimension $4 = |\mathcal{M}^c_{1, 1}(4)|$.

## r = 5

In [13]:
blocks = precompute_blocks((5, 0), fields, parity=0, precision=30);

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = D^{(s)}_{(1, 0)} = D^{(s)}_{(1, 1)} = D^{(s)}_{(\frac32, 0)} = 0$


In [14]:
fix = [
    (:s, diag_field, 1),
    (:s, Field(c, r=1//2, s=0), 0),
    (:s, Field(c, r=1, s=0), 0),
    (:s, Field(c, r=1, s=1), 0),
    (:s, Field(c, r=3//2, s=0), 0),
]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=2)
# Check that it does not converge if we fix one less constant:
println("With $(length(fix)-1) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=2);

With 5 constants fixed:
 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)    │ -4.9684e+07+6.17521e+07im  │ 9.2e-12   │
│ (3//2, -2//3)   │ -4.9684e+07+6.17521e+07im  │ 8.9e-12   │
│ (2, 0)          │ 1.55846e+12-1.09072e+12im  │ 1.7e-11   │
│ (2, -1//2)      │ -1.33301e+12+1.12162e+12im │ 1.5e-11   │
│ (2, 1//2)       │ -1.33301e+12+1.12162e+12im │ 1.2e-11   │
│ (2, 1)          │ 1.14192e+12-1.02862e+12im  │ 2e-11     │
└─────────────────┴────────────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────

This shows that the space of solutions has dimension $5 = |\mathcal{M}^c_{1, 1}(5)|$

## r = 6

In [15]:
blocks = precompute_blocks((6, 0), fields, parity=0, precision=30);

### Signature $(0, 0, 0)$, $D^{(s)}_{(1, 0)} = D^{(s)}_{(2, 0)} = D^{(s)}_{(2, \frac12)} = D^{(s)}_{(3, 0)} = D^{(s)}_{(3, \frac23)} = 0$

In [ ]:
fix = vcat(
    [(:s, diag_field, 1), (:t, diag_field, 1), (:u, diag_field, 1)],
    [[(:s, V, 0), (:t, V, 0), (:u, V, 0)] for V in [
        Field(c, r=1 // 2, s=0),
        Field(c, r=1, s=0),
        Field(c, r=1, s=1),
        Field(c, r=3 // 2, s=0),
        Field(c, r=3 // 2, s=2 // 3),
        Field(c, r=3 // 2, s=-2 // 3),
        Field(c, r=2, s=0),
    ]]...
)
# println("With $(length(fix[1:end-2])÷3) constants fixed identically in each channel:")
# solve(blocks, Sig(0, 0, 0), fix=fix[1:end-2], rmax=9//2);
println("With $(length(fix[1:end-3]) ÷ 3) constants fixed identically in each channel")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-3], rmax=2);
println("With $(length(fix[1:end]) ÷ 3) constants fixed identically in each channel")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=2);

With 7 constants fixed identically in each channel
 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)    │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)   │ 0.0+0.0im                  │ 0         │
│ (2, 0)          │ -3.8802e+28+3.46148e+28im  │ 0.35      │
│ (2, -1//2)      │ 3.99593e+28-3.4949e+28im   │ 0.35      │
│ (2, 1//2)       │ 3.99593e+28-3.4949e+28im   │ 0.35      │
│ (2, 1)          │ -4.35648e+28+3.59596e+28im │ 0.35      │
│ (5//2, 0)       │ -1.59286e+27-2.88924e+27im │ 0.99      │
│ (5//2, -2//5)   │ 1.8

In [ ]:
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=9//2);